In [10]:
import pandas as pd
import numpy as np
# load csv into pandas dataframe
search_trend_df = pd.read_csv('2020_US_weekly_symptoms_dataset.csv')

# since we have several NaN columns, we can drop the columns with null values to get better result
search_trend_df = search_trend_df.dropna(axis=1, how='all')
search_trend_df = search_trend_df.drop(['country_region_code', 'country_region', 'sub_region_1_code'], axis=1)
search_trend_df = search_trend_df.rename({'sub_region_1': 'region_name'}, axis=1)
# now, check the hospitalization cases

print(search_trend_df.info())

hospitalization_df = pd.read_csv('aggregated_cc_by.csv')

# since we have several NaN columns, we can drop the columns with null values to get better result
hospitalization_df = hospitalization_df[['open_covid_region_code', 'region_name', 'date', 'hospitalized_new']]
hospitalization_df = hospitalization_df.dropna(axis=0, how='all')

# then we need to resample the data to weekly basis
hospitalization_df['date'] = pd.to_datetime(hospitalization_df['date'], format='%Y-%m-%d')
search_trend_df['date'] = pd.to_datetime(search_trend_df['date'], format='%Y-%m-%d')

hospitalization_df = hospitalization_df.groupby(['open_covid_region_code', 'region_name'])\
    .resample('W', on='date').agg('sum').reset_index()

hospitalization_df = hospitalization_df.set_index('date')
hospitalization_df.index = hospitalization_df.index + pd.tseries.frequencies.to_offset('1d')
hospitalization_df = hospitalization_df.reset_index()
result = search_trend_df.merge(hospitalization_df, on=['open_covid_region_code', 'region_name', 'date'])
result.reset_index()
print(result.info())
result.to_csv("result.csv", index=False)
print(pd.DataFrame(result[result.columns[3:-1]].sum(axis=0), columns=['sum']).sort_values(by=['sum'], ascending=False).head(10))

# print(pd.DataFrame(result.sum(axis=0), columns=['sum']).sort_values(by=['sum']))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Columns: 124 entries, open_covid_region_code to symptom:Yawn
dtypes: float64(121), object(3)
memory usage: 558.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 433 entries, 0 to 432
Columns: 125 entries, open_covid_region_code to hospitalized_new
dtypes: datetime64[ns](1), float64(122), object(2)
memory usage: 426.2+ KB
None
                                      sum
symptom:Aphonia                   4478.88
symptom:Viral pneumonia           4136.85
symptom:Shallow breathing         3902.18
symptom:Angular cheilitis         3526.01
symptom:Dysautonomia              3435.65
symptom:Laryngitis                3280.74
symptom:Crackles                  3268.78
symptom:Ventricular fibrillation  3209.33
symptom:Rectal pain               3170.71
symptom:Hemolysis                 3168.77


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
